# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [2]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [3]:
# Initialize and constants

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [4]:
# A class to represent a Webpage

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [5]:
ed = Website("https://www.jaivikhimalay.com")
ed.links

['/cart',
 '#page',
 '/',
 '#services',
 '#about',
 '#contact',
 '/',
 '#services',
 '#about',
 '#contact',
 '#services',
 '#about',
 '#contact',
 '#services',
 'http://squarespace.com']

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [6]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [7]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [8]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [9]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://www.jaivikhimalay.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
/cart
#page
/
#services
#about
#contact
/
#services
#about
#contact
#services
#about
#contact
#services
http://squarespace.com


In [10]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [11]:
anthropic = Website("https://anthropic.com")
anthropic.links

['/',
 '/claude',
 '/team',
 '/enterprise',
 '/api',
 '/pricing',
 '/research',
 '/company',
 '/careers',
 '/news',
 'https://www.anthropic.com/research#entry:8@1:url',
 'https://www.anthropic.com/claude',
 'https://claude.ai/',
 '/api',
 '/news/3-5-models-and-computer-use',
 '/claude/sonnet',
 '/claude/haiku',
 '/news/claude-for-enterprise',
 '/research/constitutional-ai-harmlessness-from-ai-feedback',
 '/news/core-views-on-ai-safety',
 '/jobs',
 '/',
 '/claude',
 '/api',
 '/team',
 '/pricing',
 '/research',
 '/company',
 '/customers',
 '/news',
 '/careers',
 'mailto:press@anthropic.com',
 'https://support.anthropic.com/',
 'https://status.anthropic.com/',
 '/supported-countries',
 'https://twitter.com/AnthropicAI',
 'https://www.linkedin.com/company/anthropicresearch',
 'https://www.youtube.com/@anthropic-ai',
 '/legal/consumer-terms',
 '/legal/commercial-terms',
 '/legal/privacy',
 '/legal/aup',
 '/responsible-disclosure-policy',
 'https://trust.anthropic.com/']

In [12]:
get_links("https://anthropic.com")

{'links': [{'type': 'about page', 'url': 'https://anthropic.com/company'},
  {'type': 'careers page', 'url': 'https://anthropic.com/careers'},
  {'type': 'team page', 'url': 'https://anthropic.com/team'},
  {'type': 'enterprise page', 'url': 'https://anthropic.com/enterprise'},
  {'type': 'api page', 'url': 'https://anthropic.com/api'},
  {'type': 'pricing page', 'url': 'https://anthropic.com/pricing'},
  {'type': 'research page', 'url': 'https://anthropic.com/research'},
  {'type': 'news page', 'url': 'https://anthropic.com/news'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [13]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [14]:
print(get_all_details("https://anthropic.com"))

Found links: {'links': [{'type': 'about page', 'url': 'https://anthropic.com/company'}, {'type': 'careers page', 'url': 'https://anthropic.com/careers'}, {'type': 'team page', 'url': 'https://anthropic.com/team'}, {'type': 'research page', 'url': 'https://anthropic.com/research'}, {'type': 'enterprise page', 'url': 'https://anthropic.com/enterprise'}, {'type': 'customers page', 'url': 'https://anthropic.com/customers'}]}
Landing page:
Webpage Title:
Home \ Anthropic
Webpage Contents:
Claude
Overview
Team
Enterprise
API
Pricing
Research
Company
Careers
News
AI
research
and
products
that put safety at the frontier
Claude.ai
Meet Claude 3.5 Sonnet
Claude 3.5 Sonnet, our most intelligent AI model, is now available.
Talk to Claude
API
Build with Claude
Start using Claude to drive efficiency and create new revenue streams.
Learn more
Announcements
Introducing computer use, a new Claude 3.5 Sonnet, and Claude 3.5 Haiku
Oct 22, 2024
Model updates
3.5 Sonnet
3.5 Haiku
Our Work
Product
Claude fo

In [15]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [16]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:20_000] # Truncate if more than 20,000 characters
    return user_prompt

In [17]:
get_brochure_user_prompt("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://anthropic.com/company'}, {'type': 'careers page', 'url': 'https://anthropic.com/careers'}, {'type': 'team page', 'url': 'https://anthropic.com/team'}, {'type': 'research page', 'url': 'https://anthropic.com/research'}, {'type': 'enterprise page', 'url': 'https://anthropic.com/enterprise'}, {'type': 'api page', 'url': 'https://anthropic.com/api'}, {'type': 'pricing page', 'url': 'https://anthropic.com/pricing'}, {'type': 'news page', 'url': 'https://anthropic.com/news'}, {'type': 'customers page', 'url': 'https://anthropic.com/customers'}]}


"You are looking at a company called: Anthropic\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHome \\ Anthropic\nWebpage Contents:\nClaude\nOverview\nTeam\nEnterprise\nAPI\nPricing\nResearch\nCompany\nCareers\nNews\nAI\nresearch\nand\nproducts\nthat put safety at the frontier\nClaude.ai\nMeet Claude 3.5 Sonnet\nClaude 3.5 Sonnet, our most intelligent AI model, is now available.\nTalk to Claude\nAPI\nBuild with Claude\nStart using Claude to drive efficiency and create new revenue streams.\nLearn more\nAnnouncements\nIntroducing computer use, a new Claude 3.5 Sonnet, and Claude 3.5 Haiku\nOct 22, 2024\nModel updates\n3.5 Sonnet\n3.5 Haiku\nOur Work\nProduct\nClaude for Enterprise\nSep 4, 2024\nAlignment\n·\nResearch\nConstitutional AI: Harmlessness from AI Feedback\nDec 15, 2022\nAnnouncements\nCore Views on AI Safety: When, Why, What, and How\nMar 8, 2023\nWo

In [18]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [19]:
create_brochure("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://anthropic.com/company'}, {'type': 'careers page', 'url': 'https://anthropic.com/careers'}, {'type': 'team page', 'url': 'https://anthropic.com/team'}, {'type': 'research page', 'url': 'https://anthropic.com/research'}, {'type': 'enterprise page', 'url': 'https://anthropic.com/enterprise'}, {'type': 'api page', 'url': 'https://anthropic.com/api'}, {'type': 'pricing page', 'url': 'https://anthropic.com/pricing'}, {'type': 'news page', 'url': 'https://anthropic.com/news'}]}


# Anthropic Company Brochure

---

## Overview
**Anthropic** is a leading AI safety and research company based in San Francisco, California. We are dedicated to building reliable, interpretable, and steerable AI systems, with a focus on safety and meaningful impact. Our flagship AI model, **Claude 3.5 Sonnet**, represents our commitment to developing advanced AI technology that prioritizes safety and performance.

---

## Our Mission
At Anthropic, we believe in the transformative potential of AI to enhance how people interact with the world. Our goal is to develop AI systems that people can trust, leading to a safer future. We conduct pioneering research that addresses the opportunities and challenges presented by advanced AI technologies.

---

## Company Culture
### Core Values
1. **Mission-Driven**: We exist to ensure AI technology augments societal well-being.
2. **High Trust**: We foster an environment built on good faith, kindness, and honesty.
3. **Collaborative Spirit**: Teamwork is at the heart of our operations, promoting diverse contributions across all levels.
4. **Pragmatism**: We value practical, effective solutions over novel complexity.

### Work Environment
- Interdisciplinary collaborative team consisting of experts in machine learning, policy, product development, and more.
- Employees benefit from flexible paid time off, paid parental leave, and comprehensive wellness support.

---

## Research & Development
Our research efforts focus on **frontier AI technologies** and safety practices to create models that are not only effective but also controllable and interpretable. We explore diverse areas, including:
- Interpretability
- Reinforcement Learning from human feedback
- Policy impact assessments

---

## Products and Services
### Claude AI
- **Claude as a Service**: Facilitate faster decision-making and productivity across organizations.
- **Claude for Enterprise**: A secure solution for organizations wanting to leverage internal knowledge and improve team collaboration.

### Use Cases
- Automating customer support
- Assisting in document preparation
- Enhancing marketing strategies
- Accelerating creative processes

---

## Customer Engagement
We work with a variety of customers across sectors, offering tailored solutions that empower teams by:
- Scaling internal expertise and knowledge sharing.
- Producing high-quality outputs efficiently while ensuring data security.
- Leveraging AI to transform operations and improve overall productivity.

---

## Careers at Anthropic
Join an innovative team at Anthropic! We are hiring for various roles that value diverse experiences and skills. Here’s what we offer to employees:
- Competitive salaries coupled with equity packages.
- Health, dental, and vision insurance, with additional benefits catering to family planning and wellness.
- A supportive structure for relocation and home-office adjustment.

### Hiring Process
We have a rigorous yet thoughtful hiring process that emphasizes diverse perspectives and expertise. Our approach includes:
1. Exploratory chats to understand candidate interests.
2. Skills assessments tailored to role requirements.
3. Team screens to build collaborative fits.
4. Final interviews with leadership.

---

## Connect with Us
For more information, visit our website or reach out:
- **Website**: [Anthropic](https://www.anthropic.com)
- **LinkedIn**: [Anthropic LinkedIn](https://www.linkedin.com/company/anthropic)
- **Twitter**: [@Anthropic](https://twitter.com/Anthropic)

Together, let’s shape the future of safe AI!

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [20]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [47]:
stream_brochure("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://anthropic.com/company'}, {'type': 'careers page', 'url': 'https://anthropic.com/careers'}, {'type': 'team page', 'url': 'https://anthropic.com/team'}, {'type': 'research page', 'url': 'https://anthropic.com/research'}]}



# Anthropic Company Brochure

## Who We Are
Anthropic is a pioneering AI safety and research company headquartered in San Francisco. We specialize in developing reliable, interpretable, and steerable AI systems through interdisciplinary collaboration among experts in machine learning, physics, public policy, and business.

## Our Mission
At Anthropic, we aim to ensure that transformative AI technologies benefit humanity. Our research focuses on the opportunities and challenges presented by advanced AI systems, striving to create tools that people can rely upon.

## Meet Claude
We are proud to introduce **Claude**, our flagship AI model. Claude 3.5 Sonnet is our most intelligent version yet, designed to drive efficiency and creativity across industries. From automating daily tasks to enhancing collaborative efforts, Claude is here to transform workflows.

## Core Values & Company Culture
- **Mission-Driven**: Our work is centered around ensuring AI contributes positively to society.
- **High Trust**: We foster an environment where emotional maturity and intellectual openness thrive. We assume good intentions and prioritize honest communication.
- **Collaborative Spirit**: Teamwork is vital at Anthropic. We believe in crossing departmental lines to leverage our collective expertise towards common goals.
- **Pragmatism & Empiricism**: We encourage practical solutions and prioritize understanding through research and experience.

## Customer Focus
Our innovative AI tools benefit a wide range of customers, including businesses, nonprofits, and civil society organizations. We partner with clients to customize Claude's capabilities to their unique needs, enhancing productivity and insights.

## Careers at Anthropic
We are always looking for talented individuals to join our diverse team. Whether you're in research, engineering, operations, or policy, Anthropic offers a supportive environment with competitive compensation packages, comprehensive health benefits, and a strong emphasis on work-life balance.

**What We Offer:**
- Comprehensive health, dental, and vision insurance
- Flexible paid time off and absence policies
- Generous parental leave policies
- Professional development and education stipends
- A vibrant workplace culture with daily meals and wellness support

**Hiring Process:**
Our interview process is designed to minimize bias and include a thoughtful assessment of skills and team fit. We value diversity in backgrounds and experiences, and encourage candidates from all walks of life to apply.

## Join Us!
If you're as passionate about AI and its responsible development as we are, consider joining us at Anthropic. Together, we can shape the future of AI to benefit all of humanity.

---

For more information, visit [Anthropic's Website](https://www.anthropic.com) or follow us on [LinkedIn](https://www.linkedin.com/company/anthropic).


In [21]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}]}


# Hugging Face Company Brochure

---

## **Welcome to Hugging Face**
### **The AI community building the future.**

Hugging Face is revolutionizing the machine learning landscape with a collaborative platform designed for developers, researchers, and organizations. Our mission is to democratize machine learning by providing robust tools, models, and datasets that empower everyone to achieve innovation in AI.

---

## **What We Offer**
### **Models, Datasets, and Spaces**
- **400k+ Models**: Access state-of-the-art models across modalities including text, image, video, and audio.
- **100k+ Datasets**: Browse and share diverse datasets for various machine learning tasks.
- **150k+ Applications**: Explore innovative applications in our community spaces, aiding exploration and creativity.

### **Enterprise Solutions**
With enterprise-grade security and dedicated support, Hugging Face empowers organizations with a streamlined path to AI excellence.

- **Subscription Models**: Tailored plans starting at $20/user/month for robust enterprise solutions.
- **Advanced Security Features**: Benefit from Single Sign-On (SSO), audit logs, analytics, and resource management.
- **Compute Solutions**: Easily deploy and scale your ML projects with managed infrastructure facilitated by Hugging Face APIs.

---

## **Join Our Community**
- **223 Team Members**: With experts in AI, ML, and community development, our diverse team is dedicated to making quality machine learning accessible to all.
- **Collaboration at the Core**: Hugging Face encourages collaboration within our community, where everyone is invited to contribute and share knowledge.
- **Career Opportunities**: We are constantly on the lookout for passionate individuals to join our mission. If you are eager to contribute to the future of AI, check our current openings on the [Careers Page](#).

---

## **Our Culture**
At Hugging Face, we foster a culture of openness, innovation, and collaboration. We believe in:

- **Learning and Experimentation**: Encouraging a growth mindset where team members learn from each other and experiment with new ideas.
- **Community Engagement**: Our community is at the heart of everything we do. We actively seek input and insights from our users.
- **Diversity and Inclusion**: We are committed to creating an inclusive workplace where diverse backgrounds and perspectives are celebrated.

---

## **Our Customers**
More than 50,000 organizations utilize Hugging Face, including leading tech companies like:

- **Meta**
- **Amazon Web Services**
- **Google**
- **Intel**
- **Microsoft**

Join us in creating an expansive ecosystem where AI models thrive!

---

## **Get Involved**
Visit us at [Hugging Face](https://huggingface.co) to explore our resources or join our community discussions on platforms like Discord and GitHub.

Let’s build the future of AI together!

---

### **Contact Us**
- **Email**: support@huggingface.co
- **Follow Us**: 
  - Twitter: [@huggingface](https://twitter.com/huggingface)
  - LinkedIn: [Hugging Face](https://linkedin.com/company/huggingface)

--- 

*Thank you for considering Hugging Face as your partner in AI innovation!*

In [22]:
#import Gradio
import gradio as gr

In [23]:
#now extend the above to include Gradio as per Week2-day2 assignment
view = gr.Interface(
    fn=stream_brochure,
    inputs=[
        gr.Textbox(label="Company name:"),
        gr.Textbox(label="Landing page URL including http:// or https://")],
    outputs=[gr.Markdown(label="Brochure:")],
    flagging_mode="never"
)
view.launch(share=True)


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://e5232c93c091951765.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Found links: {'links': [{'type': 'about page', 'url': 'https://www.jaivikhimalay.com/#about'}, {'type': 'contact page', 'url': 'https://www.jaivikhimalay.com/#contact'}]}


# Jaivik Himalay Company Brochure

---

## About Us

Jaivik Himalay is a dynamic and innovative company situated in Silicon Valley, California. With two decades of expertise, we specialize in transforming complex issues into efficient solutions across various domains including **payments**, **gift cards**, and **logistics**. Our dedication to enhancing the accessibility of products and services drives our commitment to client success.

## Our Services

At Jaivik Himalay, we offer a comprehensive suite of services tailored for **small and medium-sized businesses**:

### Application Architecture, Design & Development
We engage with clients to automate business processes by transforming their requirements into ready-to-use products, guiding them from initial concept to product launch.

### Application Refactoring, Performance Improvement & Integration
Our expert team specializes in revitalizing outdated applications, enhancing performance and scalability to ensure they meet contemporary business needs.

### Distribution Services
We provide comprehensive solutions to help businesses efficiently source products from local and global markets, optimizing import and export logistics to lower costs.

## Our Culture

Jaivik Himalay fosters a culture of **collaboration**, **innovation**, and **continuous improvement**. Our team is composed of seasoned professionals who thrive in a dynamic environment, enabling us to adapt quickly to evolving market requirements. We believe in empowering our employees and value their contributions towards achieving our collective goals.

## Opportunities at Jaivik Himalay

### Careers
Join a team of experts committed to delivering exceptional value to our clients. Whether you are an experienced professional or just starting out, we encourage you to bring your unique skills to our organization. 

Interested candidates can reach out to us at **info@jaivikhimalay.com**.

## Contact Us

For inquiries and potential collaborations, please contact us:
- **Location**: Dublin, CA, 94568
- **Email**: info@jaivikhimalay.com

---

*Join Jaivik Himalay as we pave the way for innovative solutions and drive success for our clients.*

Found links: {'links': [{'type': 'home page', 'url': 'https://www.jaivikhimalay.com/'}, {'type': 'about page', 'url': 'https://www.jaivikhimalay.com/#about'}, {'type': 'contact page', 'url': 'https://www.jaivikhimalay.com/#contact'}]}



# Jaivik Himalay Brochure

## Company Overview
**Jaivik Himalay** is a distinguished technology and distribution firm located in **Dublin, California**, in the heart of Silicon Valley. With over **20 years** of experience, our team of seasoned experts focuses on addressing complex challenges in areas such as payments, gift cards, and logistics. We are dedicated to enhancing the accessibility of our clients' products and services, ensuring they meet the ever-evolving needs of their customers.

## Our Services
We offer a wide range of services designed to support small and medium-sized businesses throughout the product life cycle:

### 1. Application Architecture, Design & Development
We engage with clients to automate their business processes by transforming requirements into ready-to-use products. From the initial concept to the launch stage, we provide comprehensive support for developing products or APIs.

### 2. Application Refactoring, Performance Improvement & Integration
Our experienced team leverages the latest practices and technologies to revamp outdated systems, ensuring they are efficient and scalable. We redesign, test, and prepare applications for a successful relaunch.

### 3. Distribution Services
Jaivik Himalay assists companies in meeting import and export needs. We offer solutions to reduce costs and facilitate transportation both domestically and internationally, catering to a wide range of businesses.

## Company Culture
At Jaivik Himalay, we pride ourselves on fostering a collaborative and innovative environment. Our culture values:

- **Expertise:** We leverage our extensive experience to deliver optimal solutions for our clients.
- **Collaboration:** We believe in working closely with clients to understand and fulfill their needs.
- **Innovation:** We continuously adapt to industry trends and technologies to stay ahead.

## Careers
We are always on the lookout for talented individuals who share our passion for technology and excellence. Working at Jaivik Himalay offers an opportunity to be part of a dynamic team dedicated to solving real-world challenges. If you're interested in joining us, please reach out via our contact information below.

## Contact Us
For inquiries or to discuss how we can collaborate, please contact us at:

- **Email:** [info@jaivikhimalay.com](mailto:info@jaivikhimalay.com)
- **Address:** 123 Business Ave, Dublin, CA, 94568

Join us in transforming challenges into innovative solutions!

---
**© Copyright 2024 Jaivik Himalay LLC**



Found links: {'links': [{'type': 'about page', 'url': 'https://www.jaivikhimalay.com/#about'}, {'type': 'contact page', 'url': 'https://www.jaivikhimalay.com/#contact'}, {'type': 'home page', 'url': 'https://www.jaivikhimalay.com/'}]}


# Jaivik Himalay Brochure

### Company Overview
**Jaivik Himalay** is a leading service provider based in California's Silicon Valley, specializing in enhancing business processes through cutting-edge technology and distribution solutions. With over 20 years of experience, our seasoned experts are dedicated to solving complex challenges in payments, gift cards, logistics, and more.

### Services Offered
- **Application Architecture, Design & Development**: We assist small and medium-sized businesses in transforming their concepts into fully operational products. Our approach covers every stage of the product lifecycle—from design to launch.
  
- **Application Refactoring, Performance Improvement & Integration**: Our experienced team is committed to revitalizing outdated applications. We ensure applications are not only scalable but also optimized for peak performance.
  
- **Distribution Services**: We help companies optimize their import/export processes, allowing them to effectively source products locally and internationally.

### Company Culture
At Jaivik Himalay, we believe in fostering a collaborative and innovative work environment. Our culture is built on teamwork, continuous learning, and the pursuit of excellence. We encourage our team members to explore new ideas and solutions to better serve our clients and meet their evolving needs.

### Clients
We are proud to partner with a diverse range of clients, from small startups to established enterprises. Our goal is to enhance the accessibility and efficiency of our clients’ products and services, ensuring they can thrive in today's competitive landscape.

### Career Opportunities
Join our team of experts at Jaivik Himalay. If you are passionate about technology and eager to work in a dynamic environment, we invite you to be a part of our journey. We are always on the lookout for talented individuals who are ready to innovate and contribute to meaningful projects.

### Contact Us
Interested in our services or careers at Jaivik Himalay? Get in touch with us!

- **Location**: Dublin, CA, 94568
- **Email**: [info@jaivikhimalay.com](mailto:info@jaivikhimalay.com)

---

**Follow us to discover how we can help transform your business!** 

© Copyright 2024 Jaivik Himalay LLC.

Found links: {'links': [{'type': 'home page', 'url': 'https://www.jaivikhimalay.com/'}, {'type': 'about page', 'url': 'https://www.jaivikhimalay.com#about'}, {'type': 'contact page', 'url': 'https://www.jaivikhimalay.com#contact'}]}


# Jaivik Himalay Brochure

Welcome to **Jaivik Himalay**, where innovation meets accessibility in the digital landscape. Based in the heart of California's Silicon Valley, our team of seasoned experts has dedicated over 20 years to solving complex challenges in payments, logistics, gift cards, and beyond.

## Our Services

### 1. Application Architecture, Design & Development
We specialize in transforming ideas into reality. Our team provides end-to-end support for small and medium-sized businesses, from initial concept development to a successful product launch. We automate business processes by crafting ready-to-use products tailored to your needs.

### 2. Application Refactoring & Performance Improvement
Don't let outdated technology slow you down. Our experienced team can rejuvenate sluggish systems, redesign applications that lack scalability, and ensure your technology is ready for the future.

### 3. Distribution Services
We offer comprehensive support for businesses needing to source a diverse range of products, both locally and globally. Our logistics expertise allows us to reduce import/export costs, facilitating smooth transportation of goods across borders.

## About Us
At Jaivik Himalay, we pride ourselves on our collaborative culture and commitment to excellence. Our team of experts draws on two decades of experience to deliver innovative solutions and transform challenges into opportunities. We believe in continuous learning and adaptability, which empowers our workforce to embrace the evolving tech landscape.

## Our Customers
We serve a variety of industries, assisting businesses in streamlining their operations and enhancing product accessibility. Our client base includes small and medium-sized enterprises looking for effective solutions to optimize their processes and reach their market goals.

## Careers with Us
Join a dynamic team at Jaivik Himalay and take your career to the next level! We are continually on the lookout for talented individuals who are passionate about technology and eager to contribute to impactful projects. If you're ready to make a difference in the tech world, reach out to us at **info@jaivikhimalay.com**.

## Contact Us
Interested in our services or partnership opportunities? We'd love to hear from you! 

Location:  
Dublin, CA 94568  
Email: [info@jaivikhimalay.com](mailto:info@jaivikhimalay.com)

---

**© 2024 Jaivik Himalay LLC**  
*Transforming Ideas into Reality*

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 2 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!
            </span>
        </td>
    </tr>
</table>